In [2]:
%%html
<style>
table th { text-align: left!important; }
table td { text-align: left!important; }
</style>

# 第5章 ビュー（View）

## 5.1 概要

### ビューの役割
- リクエストを受け取ってレスポンスオブジェクトを返す
- 状況に応じて、  
  1. HTMLページを表示するためのコンテンツを保持したレスポンス
  1. リダイレクトするためのレスポンス
  1. エラーを通知するためのレスポンス
  などのさまざまな種類のものを作成して返す
- 特に、「HTMLを表示するためのコンテンツを保持したレスポンス」を生成する場合は、テンプレートシステムが利用される

### 正常パターンでの処理の流れ
1. URLディスパッチャからビュー関数が呼び出される
1. ビュー内で、リクエストオブジェクトから入力データを取得して型変換
1. 入力データのバリデーション（妥当性チェック）をおこなう
1. バリデーションOKの場合は、ビジネスロジック処理を続行し、画面に表示するための変数を用意
1. 遷移先画面（表示するテンプレートやリダイレクト先）を決定
1. 遷移先画面に変数の内容をレンダリング

#### `django.shortcuts`パッケージ
いろいろなタイプのレスポンスを簡単に作成するための関数のパッケージ

## 5.2 ビュー関数の書き方（関数ベース vs クラスベース）
### ビュー関数
ビュー関数とは、以下のような特徴を備えた関数
- 第1引数に`django.http.request.HttpRequest`オブジェクトを受け取る
- 戻り値として`django.http.response.HttpResponse`オブジェクトを返す

書き方としては大きく分けて以下の2通りがある
1. 関数で書く方法（関数ベース）
1. クラスで書いて（クラスベース）、ビュー関数に変換する方法

#### `django.shortcuts.render()`関数
テンプレートシステムを使ってHttpRequestオブジェクトを作成する  
作成したオブジェクトの内部には、第2引数で指定したテンプレートと第3引数にdict型で指定したコンテキストを使ってレンダリングしたHTMLコンテンツを保持する


#### `django.views.View.as_view()`関数
クラスベースのビューをビュー関数化する


#### 関数ベース
理解しやすいが、再利用しにくい

#### クラスベース
クラスを再利用することで短く書けるケースがあるが、コードの見通しが悪くなる


#### クラスビューの利点
- 汎用ビュー（Generic View）としてさまざまな用途のクラスが提供されているのでその恩恵を受けられる
- 全ての基本となる基本汎用ビュー`django.views.generic.base.View`を利用すれば見通しの良いビューが書けるし、応用も効く
- 汎用ビューのほかにも便利なMixinクラスを多重継承しておりお決まりの処理を再利用できる

## 5.3 全ての基本となる基本汎用ビュー
### `django.views.generic.base.View`
汎用ビューの基本となるクラス

### クラスベースのビューを書くときのポイント
- `GET`リクエストには`get()`、`POST`リクエストには`post()`メソッドを用意する
- テンプレートをレンダリングする場合には`django.shortcuts.render()`を使う
- リダイレクトする場合は`django,shortcuts.redirect()`を使う
  - リダイレクト先のURLは`django.urls.reverse()`を使って取得し、ハードコーディングしないようにする
- URLディスパッチャにエラーをハンドリングさせたい場合（例えば404や403エラーページを表示したいとき）は、`django.http.response.Http404`や`django.core.exceptions.PermissionDenied`等の特別な例外をraiseして伝播させる
- `as_view()`メソッドを使ってビュー関数化する

## 5.4 シンプルでよく使う基本汎用ビュー

### 5.4.1 テンプレートの表示に特化した基本汎用ビュー（TemplateView）
#### `django.views.generic.TemplateView`クラス
`django.views.generic.base.View`クラスをベースにして、テンプレートを表示することに特化した基本汎用ビュークラス  
トップ画面やヘルプ画面などの単純なテンプレートを表示するのによく利用される  
このクラスを使ったビューは非常に簡潔にかけるため、URLconf内に直接書いてしまう場合もある

##### `TemplateView`を継承したビューの例
~~~python
from django.views.generic import TemplateView

classIndexView(TemplateView):
    template_name = 'index.html'
    
index = IndexView.as_view()
~~~

##### `get_context_data()`メソッド
ビューに表示するコンテキスト変数を取得するメソッド  
TemplateViewを元にしたクラスベースのビューの場合は、このメソッドをオーバーライドすることで、テンプレートにわたすコンテキストに任意の変数を追加することが出来る。

###### `get_context_data()`メソッドをオーバーライドした例
~~~python
from django.contrib.auth.models import User
from django.views.generic import TemplateView

class IndexView(TemplateView):
    template_name = 'index.html'
    
    def get_context_data(self, **kwards):
        context = super(IndexView, self).get_context_data(**kwards)
        # テンプレートにわたすコンテキストに「user_count」という変数を追加
        context['user_count'] = User.objects.all().count()
        return context
    
index = IndexView.as_view()
~~~


### 5.4.2 リダイレクトに特化した基本汎用ビュー（RedirectView）
#### `django.views.generic.RedirectView`クラス
任意のURLにリダイレクトすることに特化した基本汎用ビュークラス  
このクラスを使ったビューも、`TemplateView`と同様にURLconf内に直接書いてしまう場合がある

##### `RedirectView`を使ったビューの例
~~~python
from django.views.generic import RedirectView

class IndexView(RedirectView):
    url = '/accounts/login/'
    
index = IndexView.as_view()
~~~

##### `url`クラス変数
リダイレクト先のURLを指定する変数

##### `pattern_name`クラス変数
リダイレクト先のURLをURLconfに登録されたパターンから取得するためのパターン名を指定する変数

##### `get_redirect_url`メソッド
リダイレクトするURLを動的に組み立てるメソッド  
RedirectViewを元にしたクラスベースのビューでリダイレクト先を動的に組み立てる場合は、このメソッドをオーバーライドする

## 5.5 様々な用途に特化した汎用ビュー

### 代表的な汎用ビュークラス

| 汎用ビュークラス | 説明 |
| :--------------- | :--- |
| `django.views.generic.list.ListView` | モデルオブジェクトの一覧を表示するためのView |
| `django.views.generic.detail.DetailView` | モデルオブジェクトの詳細を表示するためのView |
| `django.views.generic.edit.FormView` | フォームを利用するためのView |
| `django.views.generic.edit.CreateView` | モデルオブジェクトを登録するためのView |
| `django.views.generic.edit.UpdateView` | モデルオブジェクトを更新するためのView |
| `django.views.generic.edit.DeleteView` | モデルオブジェクトを削除するためのView |

#### `ListView`を使ったビューの例
~~~python
from django.views.generic import ListView

class BookListView(ListView):
    model = Book
~~~

## 5.6 ログイン・ログアウトについて

### 5.6.1 ログイン処理について
#### `django.contrib.auth.authenticate()`メソッド
ユーザー名とパスワードを使った認証を行うためのメソッド

#### `django.contrib.auth.login()`メソッド
ユーザーセッションの管理を行うためのメソッド  
実行すると以下の処理が行われる
- セッションにランダムに生成された「セッションID」をキーにしてユーザーの情報（ユーザーIDやオブジェクトの格納先）が保存される
- シグナルが実行され、Userオブジェクトに対応するレコードの最終更新日時が更新される

その後、`SessionMiddleware`によってCookieの`sessionid`キーにセッションIDが保管される

#### ログイン後の処理
`AuthenticationMiddleware`により、リクエストのたびにCookieからセッションIDを取り出し、それを元にセッションからユーザーの情報を取り出し、さらにそれを元にしてUserオブジェクトを取得して、リクエストオブジェクトの`user`属性にセットする  
ログインしていない場合は`AnonymousUser`という特別なクラスのオブジェクトが`user`属性にセットされる


#### ユーザーがログイン済であるかの判定
`is_authenticated`属性を使う 


### 5.6.2 ログアウト処理について
#### `django.contrib.auth.logout()`メソッド
ログアウト処理を行うメソッド

#### ログアウト時の処理
サーバ側のセッションがクリアされ、リクエストオブジェクトの`user`属性に`AnonymousUser`オブジェクトがセットされる


### 5.6.3 LoginView/LogoutViewについて
#### ログイン/ログアウトに特化したビュー
- `django.contrib.auth.views.LoginView`
  ログインに特化したビュー
- `django.contrib.auth.views.LogoutView`
  ログアウトに特化したビュー

#### `LoginView`を使ったビューの例
~~~python
from django.contrib.auth.views import LoginView as AuthLoginView

class LoginView(AuthLoginView):
    template_name = 'accounts/login.html'
    
login = LoginView.as_view()
~~~

#### `LoginView`および`LogoutView`で利用される変数一覧
| 変数名 | 説明 | デフォルト値 |
| :----- | :--- | :----------- |
| `LOGIN_URL` | ログインページのURL | `/accounts/login/` |
| `LOGIN_REDIRECT_URL` | ログイン後のリダイレクト先 | `/accounts/profile/` |
| `LOGOUT_REDIRECT_URL` | ログアウト後のリダイレクト先 | None |

### 5.6.4 `LoginRequiredMixin`について
#### `django.contrib.auth.mixins.LoginRequiredMixin`
未ログインのユーザーがアクセスしようとしたときに何らかのペナルティを課すためのMixin  
Django 1.9から導入　
クラスベースのビューの場合にこのクラスを継承して作成すると、未ログインのユーザーがアクセスしたときに`LOGIN_URL`で定義したURLにリダイレクトしてくれるようになる  
`raise_exception = True`の記述をすると、403エラーの画面を表示することも可能

#### `LoginRequiredMixin`を利用したビューの例
~~~python
from django.contrib.auth.mixins import LoginRequiredMixin
from django.views.generic import ListView

class BookListView(LoginRequiredMixin, ListView):
    
    model = Book
    # 403エラー画面を表示する場合は次のコメントアウトを外す
    # raise_exception = True
~~~

## 5.7 まとめ
- ビューは`views.py`に書く
- レスポンスオブジェクトを作成して返すのがビューの基本
  - テンプレートを使う場合は`django.shortcuts.render()`を使う
  - リダイレクトする場合は`django.shortcuts.redirect()`を使う
- 関数ベースではなく、クラスベースのビューを使おう
  - まずは全ての基本となる`django.views.generic.base.View`を継承したビューの書き方に慣れよう
- ビューに複雑なロジックは書かず、見通しをよくしよう